# Application of Bootstrap samples in Random Forest

In [1]:
import numpy as np
from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error

 <li> Load the boston house dataset </li>

In [2]:
boston = load_boston()
x=boston.data #independent variables
y=boston.target #target variable

In [3]:
x.shape

(506, 13)

In [20]:
x[[1,2,3],:]#delete after use

array([[2.7310e-02, 0.0000e+00, 7.0700e+00, 0.0000e+00, 4.6900e-01,
        6.4210e+00, 7.8900e+01, 4.9671e+00, 2.0000e+00, 2.4200e+02,
        1.7800e+01, 3.9690e+02, 9.1400e+00],
       [2.7290e-02, 0.0000e+00, 7.0700e+00, 0.0000e+00, 4.6900e-01,
        7.1850e+00, 6.1100e+01, 4.9671e+00, 2.0000e+00, 2.4200e+02,
        1.7800e+01, 3.9283e+02, 4.0300e+00],
       [3.2370e-02, 0.0000e+00, 2.1800e+00, 0.0000e+00, 4.5800e-01,
        6.9980e+00, 4.5800e+01, 6.0622e+00, 3.0000e+00, 2.2200e+02,
        1.8700e+01, 3.9463e+02, 2.9400e+00]])

### Task: 1
<font color='red'><b>Step 1 Creating samples: </b></font> Randomly create 30 samples from the whole boston data points.
<ol>
<li>Creating each sample: Consider any random 303(60% of 506) data points from whole data set and then replicate any 203 points from the sampled points</li>
<li>Ex: For better understanding of this procedure lets check this examples, assume we have 10 data points [1,2,3,4,5,6,7,8,9,10], first we take 6 data points randomly consider we have selected [4, 5, 7, 8, 9, 3] now we will replciate 4 points from [4, 5, 7, 8, 9, 3], consder they are [5, 8, 3,7] so our final sample will be [4, 5, 7, 8, 9, 3, 5, 8, 3,7]</li>
<li> we create 30 samples like this </li>
<li> Note that as a part of the Bagging when you are taking the random samples make sure each of the sample will have                different set of columns</li>
<li> Ex: assume we have 10 columns for the first sample we will select [3, 4, 5, 9, 1, 2] and for the second sample [7, 9, 1, 4, 5, 6, 2] and so on...</li>
<li> Make sure each sample will have atleast 3 feautres/columns/attributes</li>
</ol>

<font color='red'><b>Step 2 Building High Variance Models on each of the sample and finding train MSE value:</b></font> Build a DecisionTreeRegressor on each of the sample.
<ol><li>Build a regression trees on each of 30 samples.</li>
<li>computed the predicted values of each data point(506 data points) in your corpus.</li>
<li> predicted house price of $i^{th}$ data point $y^{i}_{pred} =  \frac{1}{30}\sum_{k=1}^{30}(\text{predicted value of } x^{i} \text{ with } k^{th} \text{ model})$.</li>
<li>Now calculate the $MSE =  \frac{1}{506}\sum_{i=1}^{506}(y^{i} - y^{i}_{pred})^{2}$.</li>
</ol>

<font color='red'><b>Step 3 Calculating the OOB score :</b></font>
<ol>
<li>Computed the predicted values of each data point(506 data points) in your corpus.</li>
<li>Predicted house price of $i^{th}$ data point $y^{i}_{pred} =  \frac{1}{k}\sum_{\text{k= model which was buit on samples not included } x^{i}}(\text{predicted value of } x^{i} \text{ with } k^{th} \text{ model})$.</li>
<li>Now calculate the $OOB Score =  \frac{1}{506}\sum_{i=1}^{506}(y^{i} - y^{i}_{pred})^{2}$.</li>
</ol>

### Task: 2
<pre>
<font color='red'><b>Computing CI of OOB Score and Train MSE</b></font>
<ol>
<li> Repeat Task 1 for 35 times, and for each iteration store the Train MSE and OOB score </li>
<li> After this we will have 35 Train MSE values and 35 OOB scores </li>
<li> using these 35 values (assume like a sample) find the confidence intravels of MSE and OOB Score </li>
<li> you need to report CI of MSE and CI of OOB Score </li>
<li> Note: Refer the Central_Limit_theorem.ipynb to check how to find the confidence intravel</li>
</ol>
</pre>
### Task: 3
<pre>
<font color='red'><b>Given a single query point predict the price of house.</b></font>

<li>Consider xq= [0.18,20.0,5.00,0.0,0.421,5.60,72.2,7.95,7.0,30.0,19.1,372.13,18.60] Predict the house price for this point as mentioned in the step 2 of Task 1. </li>
</pre>

In [96]:
import random 

row_indices_iob = []
row_indices_oob = []
col_indices     = []

for i in range(30):
    iob_indices = np.random.choice(range(506),303)
    iob_indices = np.hstack([iob_indices,np.random.choice(iob_indices,size=203)])
    col_index = np.array(random.sample(range(13),random.randint(3,13)))
    row_indices_iob.append(iob_indices) 
    col_indices.append(col_index)
    oob_indices = np.setdiff1d(range(506),iob_indices)
    row_indices_oob.append(oob_indices)
print(row_indices_iob[29].shape)
print(row_indices_oob[29].shape)
print(col_indices[29].shape)

(506,)
(288,)
(13,)


In [97]:
from sklearn.tree import DecisionTreeRegressor
Y_pred = [0 for i in range(506)]
for i in range(30):
    clf = DecisionTreeRegressor()
    X_train = x[row_indices_iob[i][:,None],col_indices[i]]
    Y_train = y[row_indices_iob[i]]
    clf.fit(X_train,Y_train)
    Y_pred += clf.predict(x[:,col_indices[i]])
Y_pred = Y_pred/30

errors = (y-Y_pred)**2
MSE = errors.sum()/506
print(MSE)

4.030107150427898


In [98]:
Y_pred_OOB = [0 for i in range(506)]
Ks = [0 for i in range(506)]

for i in range(30):
    k = 0
    clf = DecisionTreeRegressor()
    X_train = x[row_indices_iob[i][:,None],col_indices[i]]
    Y_train = y[row_indices_iob[i]]
    clf.fit(X_train,Y_train)

    for j in range(506):
        if j not in row_indices_iob[i]:
            Ks[j] += 1
            Y_pred_OOB[k] += clf.predict(x[j,col_indices[i]].reshape(1,-1)).item()
        k += 1
        
Y_pred_OOB = [x/y for x,y in zip(Y_pred_OOB,Ks)]

errors_OOB = (y-Y_pred_OOB)**2
OOB = errors_OOB.sum()/506
print(OOB)

11.432277641526976


In [ ]:
MSEs = []
OOBs = []
for _ in range(35):
    row_indices_iob = []
    row_indices_oob = []
    col_indices     = []

    for i in range(30):
        iob_indices = np.random.choice(range(506),303)
        iob_indices = np.hstack([iob_indices,np.random.choice(iob_indices,size=203)])
        col_index = np.array(random.sample(range(13),random.randint(3,13)))
        row_indices_iob.append(iob_indices) 
        col_indices.append(col_index)
        oob_indices = np.setdiff1d(range(506),iob_indices)
        row_indices_oob.append(oob_indices)
    
    Y_pred = [0 for i in range(506)]
    for i in range(30):
        clf = DecisionTreeRegressor()
        X_train = x[row_indices_iob[i][:,None],col_indices[i]]
        Y_train = y[row_indices_iob[i]]
        clf.fit(X_train,Y_train)
        Y_pred += clf.predict(x[:,col_indices[i]])
    Y_pred = Y_pred/30

    errors = (y-Y_pred)**2
    MSE = errors.sum()/506
    MSEs.append(MSE)
    
    Y_pred_OOB = [0 for i in range(506)]
    Ks = [0 for i in range(506)]

    for i in range(30):
        k = 0
        clf = DecisionTreeRegressor()
        X_train = x[row_indices_iob[i][:,None],col_indices[i]]
        Y_train = y[row_indices_iob[i]]
        clf.fit(X_train,Y_train)

        for j in range(506):
            if j not in row_indices_iob[i]:
                Ks[j] += 1
                Y_pred_OOB[k] += clf.predict(x[j,col_indices[i]].reshape(1,-1)).item()
            k += 1

    Y_pred_OOB = [x/y for x,y in zip(Y_pred_OOB,Ks)]

    errors_OOB = (y-Y_pred_OOB)**2
    OOB = errors_OOB.sum()/506
    OOBs.append(OOB)
    
print(MSEs)
print(OOBs)